In [19]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [20]:
import requests
import io
import urllib.parse
import json

#Command to start the DB sever and restore from file
#sudo docker run -t -d  -p 9000:9000  -v "$(pwd)/questdb_data:/root/.questdb/" questdb/questdb
#sudo docker logs --follow CONTAINER

In [3]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_driver = os.path.join(os.getcwd(), "chromedriver")
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)

In [43]:
def weather_synopsis(location):
    #rw=spotWx.get_rolling_window(2,location)
    df=location.forecast
    new_snow=df["SQP"].iloc[-1]#+rw.snowfall.sum#get total forecast and past days snow
    melting=df["TMP"].max()>0# | (rw.maxTemp>0.any()) #Melting in window or in the forecast?
    wind=df["WS"].max()
    #sun during the day hours only
    cloud=df.CLOUD.mask((df.TIME.str.slice(stop=2).astype(int)<8) | (df.TIME.str.slice(stop=2).astype(int)>20))
    sun=100-cloud.mean()
    data = {'New Snow':  [new_snow],
            'Melting': [melting],
            'Max Winds': [wind],
            'Weighted Sun':[sun]
           }
    result = pd.DataFrame(data, index=[location.name])
    return result


In [67]:
class location:
    def __init__(self,name,lat,long):
        self.name=name
        self.lat=lat
        self.long=long
        
class datasource:
    def __init__(self,name,url):
        self.urlstring=url
        self.name=name
                #For creating the table initially
        q = 'create table '+self.name+\
            '(location string,'\
            'Date string,'\
            'snowfall float,'\
            'maxTemp float,'\
            'maxWind float,'\
            'sun_percent float)'
        r = requests.get("http://localhost:9000/exec?query=" + q)
        print (r.text)
        
    def get_forecast(self,location):
        driver.get(self.urlstring.format(location.lat,location.long))
        driver.execute_script("$('.buttons-csv').click();")
        time.sleep(2)
        df=pd.read_csv("SpotWx.csv")
        os.remove("SpotWx.csv")
        setattr(location, "forecast", df)
        return True
    
    def store_today(self,location):
        #Check to see if we already have the data
        df=location.forecast
        today=df.iloc[0:df[df.TIME == '00:00'].index[0]]
        q = "select count"\
        " from "+self.name+\
        " where date ='"+today.DATE + "' and location ='"+location.name+"'"
        query = urllib.parse.quote(q[0])
        r = requests.get("http://localhost:9000/exec?query="+query)
        r=json.loads(r.text)
        if (r['dataset'][0][0]==0):
            print("Storing {} to {}".format(location.name,self.name))
            cloud=today.CLOUD.mask((today.TIME.str.slice(stop=2).astype(int)<8) | (today.TIME.str.slice(stop=2).astype(int)>20))
            sun=100-cloud.mean()
            date=today.DATE[0]
            query = "insert into "+self.name+" values('"\
            + location.name + "','"\
            + date + "',"\
            + str(today["SQP"].iloc[-1]) + ","\
            + str(today.TMP.max()) + ","\
            + str(today.WS.max()) + ","\
            + str(sun) + ")"
            r = requests.get("http://localhost:9000/exec?query=" + query)

            if (r.status_code==200):
                return True
            else:
                return False
            
        def get_rolling_window(self,window,location):
            
            return df

In [68]:
blackcomb=location("black",50.09970, -122.86943)

garibaldi=location("gar",49.93299, -123.03600)

joffre_lake=location("jof",50.34143, -122.47768)

s2s=location("s2s",9.65218, -123.09211)

cloudburst=location("cloud",49.93298, -123.23602)

manning=location("man",49.06805, -120.91646)

coq=location("coq",49.61094, -121.05758)

rogers=location("rog",51.30123, -117.52014)

sites=[blackcomb,garibaldi,joffre_lake,cloudburst,manning,coq,rogers]

In [69]:
view=pd.DataFrame(columns = ['New Snow', 'Melting','Max Winds','Weighted Sun'])

In [72]:
print("Grabbing site data...")
spotWx=datasource('spotWx',"https://spotwx.com/products/grib_index.php?model=gem_lam_continental&lat={}&lon={}&tz=America/Vancouver&display=table")
for site in sites:
    spotWx.get_forecast(site)
    spotWx.store_today(site)
    result=weather_synopsis(site)
    view=view.append(result)

Grabbing site data...
{"query":"create table spotWx(location string,Date string,snowfall float,maxTemp float,maxWind float,sun_percent float)","error":"table already exists","position":13}


/tmp/ipykernel_2796/3491188611.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  view=view.append(result)
/tmp/ipykernel_2796/3491188611.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  view=view.append(result)
/tmp/ipykernel_2796/3491188611.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  view=view.append(result)
/tmp/ipykernel_2796/3491188611.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  view=view.append(result)
/tmp/ipykernel_2796/3491188611.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  view=view.append(result)
/tmp/ipykernel_

In [65]:
view
view.style.background_gradient(cmap="RdYlGn",subset=["New Snow"]) #style snow column

,New Snow,Melting,Max Winds,Weighted Sun
black,1.000000,0,5,91.703704
gar,1.000000,0,10,90.592593
jof,0.800000,0,7,91.259259
cloud,0.600000,1,5,91.888889
man,2.100000,0,6,86.592593
coq,0.100000,0,9,86.370370
rog,1.300000,0,10,73.629630


In [66]:
view.style.background_gradient(cmap="cividis",subset=["Weighted Sun"],vmin=50) #style sun column

,New Snow,Melting,Max Winds,Weighted Sun
black,1.000000,0,5,91.703704
gar,1.000000,0,10,90.592593
jof,0.800000,0,7,91.259259
cloud,0.600000,1,5,91.888889
man,2.100000,0,6,86.592593
coq,0.100000,0,9,86.370370
rog,1.300000,0,10,73.629630


In [190]:
driver.quit()

In [73]:
#Backup the DB to file
requests.get("http://localhost:9000/exec?query=BACKUP table spotWx;")

<Response [200]>